<a href="https://colab.research.google.com/github/Faithful777/GSOCnotebooks/blob/main/Plant_CLEF_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

plantclef_2025_path = kagglehub.competition_download('plantclef-2025')

print('Data source import complete.')


Data source import complete.


In [3]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [4]:
df=pd.read_csv('/kaggle/input/plantclef-2025/PlantCLEF2024_single_plant_training_metadata.csv',sep=';')
print(len(df))
display(df[0:1].T)

/tmp/ipykernel_77/2559118519.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/kaggle/input/plantclef-2025/PlantCLEF2024_single_plant_training_metadata.csv',sep=';')


1408033


,0
image_name,59feabe1c98f06e7f819f73c8246bd8f1a89556b.jpg
organ,leaf
species_id,1396710
obs_id,1008726402
license,cc-by-sa
partner,NaN
author,Gulyás Bálint
altitude,205.9261
latitude,47.59216
longitude,19.362895


In [5]:
class_counts = df['species_id'].value_counts()
valid_classes = class_counts[class_counts >= 1].index
data2 = df[df['species_id'].isin(valid_classes)]
display(data2)
class_names=sorted(data2['species_id'].unique().tolist())
num_classes=len(class_names)
print(num_classes)

class_df=pd.DataFrame(data=class_names)
class_df.to_csv('class_names.csv',index=False)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,image_name,organ,species_id,obs_id,license,partner,author,altitude,latitude,longitude,gbif_species_id,species,genus,family,dataset,publisher,references,url,learn_tag,image_backup_url
0,59feabe1c98f06e7f819f73c8246bd8f1a89556b.jpg,leaf,1396710,1008726402,cc-by-sa,NaN,Gulyás Bálint,205.9261,47.592160,19.362895,5284517.0,Taxus baccata L.,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/59feabe1c98f06...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
1,dc273995a89827437d447f29a52ccac86f65476e.jpg,leaf,1396710,1008724195,cc-by-sa,NaN,vadim sigaud,323.7520,47.906703,7.201746,5284517.0,Taxus baccata L.,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/dc273995a89827...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
2,416235e7023a4bd1513edf036b6097efc693a304.jpg,leaf,1396710,1008721908,cc-by-sa,NaN,fil escande,101.3160,48.826774,2.352774,5284517.0,Taxus baccata L.,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/416235e7023a4b...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
3,cbd18fade82c46a5c725f1f3d982174895158afc.jpg,leaf,1396710,1008699177,cc-by-sa,NaN,Desiree Verver,5.1070,52.190427,6.009677,5284517.0,Taxus baccata L.,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/cbd18fade82c46...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
4,f82c8c6d570287ebed8407cefcfcb2a51eaaf56e.jpg,leaf,1396710,1008683100,cc-by-sa,NaN,branebrane,165.3390,45.794739,15.965862,5284517.0,Taxus baccata L.,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/f82c8c6d570287...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408028,1733a9ed3fd963c522098111cbfb493690fa8389.jpg,branch,1361255,4436105257,http://creativecommons.org/licenses/by-nc/4.0/,NaN,Patrick Le Mao,NaN,NaN,NaN,2863938.0,Zostera noltii Hornem.,Zostera,Zosteraceae,gbif,iNaturalist,https://www.inaturalist.org/photos/328489675,https://inaturalist-open-data.s3.amazonaws.com...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
1408029,b032ee147bdb01a64d499b20953227e8c6691520.jpg,habit,1361255,4436105257,http://creativecommons.org/licenses/by-nc/4.0/,NaN,Patrick Le Mao,NaN,NaN,NaN,2863938.0,Zostera noltii Hornem.,Zostera,Zosteraceae,gbif,iNaturalist,https://www.inaturalist.org/photos/328489656,https://inaturalist-open-data.s3.amazonaws.com...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
1408030,95853343b1264a110959a8f05427259ac9eccf06.jpg,habit,1361255,4436105257,http://creativecommons.org/licenses/by-nc/4.0/,NaN,Patrick Le Mao,NaN,NaN,NaN,2863938.0,Zostera noltii Hornem.,Zostera,Zosteraceae,gbif,iNaturalist,https://www.inaturalist.org/photos/328489546,https://inaturalist-open-data.s3.amazonaws.com...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...
1408031,07b12082e8cabda3986ed444bd4c94409f8de6b5.jpg,habit,1361255,4436105257,http://creativecommons.org/licenses/by-nc/4.0/,NaN,Patrick Le Mao,NaN,NaN,NaN,2863938.0,Zostera noltii Hornem.,Zostera,Zosteraceae,gbif,iNaturalist,https://www.inaturalist.org/photos/328489733,https://inaturalist-open-data.s3.amazonaws.com...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...


7806


In [6]:
N=list(range(len(class_names)))
normal_mapping=dict(zip(class_names,N))
reverse_mapping=dict(zip(N,class_names))
data2['label']=data2['species_id'].map(normal_mapping)
display(data2)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,image_name,organ,species_id,obs_id,license,partner,author,altitude,latitude,longitude,...,species,genus,family,dataset,publisher,references,url,learn_tag,image_backup_url,label
0,59feabe1c98f06e7f819f73c8246bd8f1a89556b.jpg,leaf,1396710,1008726402,cc-by-sa,NaN,Gulyás Bálint,205.9261,47.592160,19.362895,...,Taxus baccata L.,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/59feabe1c98f06...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,4826
1,dc273995a89827437d447f29a52ccac86f65476e.jpg,leaf,1396710,1008724195,cc-by-sa,NaN,vadim sigaud,323.7520,47.906703,7.201746,...,Taxus baccata L.,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/dc273995a89827...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,4826
2,416235e7023a4bd1513edf036b6097efc693a304.jpg,leaf,1396710,1008721908,cc-by-sa,NaN,fil escande,101.3160,48.826774,2.352774,...,Taxus baccata L.,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/416235e7023a4b...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,4826
3,cbd18fade82c46a5c725f1f3d982174895158afc.jpg,leaf,1396710,1008699177,cc-by-sa,NaN,Desiree Verver,5.1070,52.190427,6.009677,...,Taxus baccata L.,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/cbd18fade82c46...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,4826
4,f82c8c6d570287ebed8407cefcfcb2a51eaaf56e.jpg,leaf,1396710,1008683100,cc-by-sa,NaN,branebrane,165.3390,45.794739,15.965862,...,Taxus baccata L.,Taxus,Taxaceae,plantnet,plantnet,https://identify.plantnet.org/fr/k-southwester...,https://bs.plantnet.org/image/o/f82c8c6d570287...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,4826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408028,1733a9ed3fd963c522098111cbfb493690fa8389.jpg,branch,1361255,4436105257,http://creativecommons.org/licenses/by-nc/4.0/,NaN,Patrick Le Mao,NaN,NaN,NaN,...,Zostera noltii Hornem.,Zostera,Zosteraceae,gbif,iNaturalist,https://www.inaturalist.org/photos/328489675,https://inaturalist-open-data.s3.amazonaws.com...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,1864
1408029,b032ee147bdb01a64d499b20953227e8c6691520.jpg,habit,1361255,4436105257,http://creativecommons.org/licenses/by-nc/4.0/,NaN,Patrick Le Mao,NaN,NaN,NaN,...,Zostera noltii Hornem.,Zostera,Zosteraceae,gbif,iNaturalist,https://www.inaturalist.org/photos/328489656,https://inaturalist-open-data.s3.amazonaws.com...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,1864
1408030,95853343b1264a110959a8f05427259ac9eccf06.jpg,habit,1361255,4436105257,http://creativecommons.org/licenses/by-nc/4.0/,NaN,Patrick Le Mao,NaN,NaN,NaN,...,Zostera noltii Hornem.,Zostera,Zosteraceae,gbif,iNaturalist,https://www.inaturalist.org/photos/328489546,https://inaturalist-open-data.s3.amazonaws.com...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,1864
1408031,07b12082e8cabda3986ed444bd4c94409f8de6b5.jpg,habit,1361255,4436105257,http://creativecommons.org/licenses/by-nc/4.0/,NaN,Patrick Le Mao,NaN,NaN,NaN,...,Zostera noltii Hornem.,Zostera,Zosteraceae,gbif,iNaturalist,https://www.inaturalist.org/photos/328489733,https://inaturalist-open-data.s3.amazonaws.com...,train,https://lab.plantnet.org/LifeCLEF/PlantCLEF202...,1864


In [ ]:
# Get the first 10 rows
first_10 = data2.head(10)

# Save to a new CSV file
first_10.to_csv('first_10_rows.csv', index=False)

In [9]:
import pandas as pd
import numpy as np
import math
import random
from scipy.sparse import lil_matrix, save_npz
from concurrent.futures import ThreadPoolExecutor, as_completed

# === Constants ===
EARTH_RADIUS_KM = 6371.0088
GRID_AREA_KM2 = 0.5
SPECIES_COUNT = 7806

def calculate_grid_size(area_km2):
    side_length_km = math.sqrt(area_km2)
    return side_length_km / 110.574

def random_offset(grid_size):
    return random.uniform(0, grid_size)

def get_grid_edges(lat, lon, grid_size, lat_offset, lon_offset):
    lat_shifted = lat + lat_offset
    lon_shifted = lon + lon_offset

    lat_grid_idx = math.floor(lat_shifted / grid_size)
    lon_grid_idx = math.floor(lon_shifted / grid_size)

    lat_min = (lat_grid_idx * grid_size) - lat_offset
    lat_max = lat_min + grid_size
    lon_min = (lon_grid_idx * grid_size) - lon_offset
    lon_max = lon_min + grid_size

    return (lat_min, lat_max, lon_min, lon_max)

def parallel_grid_assignment(df_chunk, grid_size, lat_offset, lon_offset):
    grid_edges_list = []
    species_map = {}

    for _, row in df_chunk.iterrows():
        edges = get_grid_edges(row['latitude'], row['longitude'], grid_size, lat_offset, lon_offset)
        grid_edges_list.append(edges)
        species_map.setdefault(edges, set()).add(int(row['label']))
    return grid_edges_list, species_map

def parallel_label_assignment(indexed_rows, species_in_grid, grid_size, lat_offset, lon_offset):
    from scipy.sparse import lil_matrix
    result = []
    for i, row in indexed_rows:
        edges = get_grid_edges(row.latitude, row.longitude, grid_size, lat_offset, lon_offset)
        species_set = species_in_grid[edges]
        indices = list(species_set)
        result.append((i, indices))
    return result

def main():
    #df = pd.read_csv('first_10_rows (1).csv')
    df = data2
    df = df.dropna(subset=['latitude', 'longitude'])
    grid_size = calculate_grid_size(GRID_AREA_KM2)
    lat_offset = random_offset(grid_size)
    lon_offset = random_offset(grid_size)

    # Split data for parallel grid edge assignment
    num_threads = 8
    df_chunks = np.array_split(df, num_threads)

    species_in_grid = {}
    grid_edges_list = []

    # Threaded calculation for grid edges and species map
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(parallel_grid_assignment, chunk, grid_size, lat_offset, lon_offset)
                   for chunk in df_chunks]

        for future in as_completed(futures):
            edges_part, species_part = future.result()
            grid_edges_list.extend(edges_part)
            for k, v in species_part.items():
                species_in_grid.setdefault(k, set()).update(v)

    grid_edges_array = np.array(grid_edges_list)

    # Sparse matrix for labels
    label_vectors_sparse = lil_matrix((len(df), SPECIES_COUNT), dtype=int)

    indexed_rows = list(enumerate(df.itertuples(index=False)))

    # Threaded label vector assignment
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for i in range(num_threads):
            chunk = indexed_rows[i::num_threads]
            futures.append(executor.submit(
                parallel_label_assignment, chunk, species_in_grid, grid_size, lat_offset, lon_offset
            ))

        for future in as_completed(futures):
            label_data = future.result()
            for i, species_indices in label_data:
                label_vectors_sparse.rows[i] = species_indices
                label_vectors_sparse.data[i] = [1] * len(species_indices)

    # Save results
    np.save('grid_edges.npy', grid_edges_array)
    save_npz('label_vectors_sparse.npz', label_vectors_sparse.tocsr())

    print("Grid edges shape:", grid_edges_array.shape)
    print("Label vectors saved as sparse CSR matrix.")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Grid edges shape: (705424, 4)
Label vectors saved as sparse CSR matrix.


In [32]:
# load grid edges array
grid_edges_array = np.load('grid_edges.npy')
print(grid_edges_array.shape)

(705424, 4)


In [10]:
from scipy.sparse import load_npz

# Load the saved sparse matrix
label_vectors_sparse = load_npz('label_vectors_sparse.npz')

# Confirm its shape
print(label_vectors_sparse.shape)


(705424, 7806)


In [31]:
# Get the 5th label vector (still sparse)
label_vector_sparse = label_vectors_sparse[4]

# Convert to dense array if needed
label_vector_dense = label_vector_sparse.toarray().flatten()

print(label_vector_dense.max())


1


In [27]:
# Get nonzero indices directly (species IDs)
species_present = label_vectors_sparse[2000].nonzero()[1]
print("Species IDs present:", species_present)
print("Number of species present:", len(species_present))

Species IDs present: [ 195 1796 1477 4745 2571 3915 2062 3858 3349 4375 4056 1563 5852 1949
 2937 1259 6123 1517 5935 5233 4025 2747 3836]
Number of species present: 23


In [12]:
from scipy.sparse import load_npz

# Load the sparse matrix
label_vectors_sparse = load_npz('label_vectors_sparse.npz')

# Calculate species count per point (non-zero entries per row)
species_counts = label_vectors_sparse.getnnz(axis=1)

# Calculate average
average_species_per_grid = species_counts.mean()

print(f"Average number of species in a grid: {average_species_per_grid:.2f}")


Average number of species in a grid: 10.23


In [ ]:
dir0=df['image_backup_url'].tolist()
print(dir0[0:3])

['https://lab.plantnet.org/LifeCLEF/PlantCLEF2024/single_plant_training_data/PlantCLEF2024singleplanttrainingdata/train/1396710/59feabe1c98f06e7f819f73c8246bd8f1a89556b.jpg', 'https://lab.plantnet.org/LifeCLEF/PlantCLEF2024/single_plant_training_data/PlantCLEF2024singleplanttrainingdata/train/1396710/dc273995a89827437d447f29a52ccac86f65476e.jpg', 'https://lab.plantnet.org/LifeCLEF/PlantCLEF2024/single_plant_training_data/PlantCLEF2024singleplanttrainingdata/train/1396710/416235e7023a4bd1513edf036b6097efc693a304.jpg']


In [ ]:
def create_path_label_list(df):
    path_label_list = []
    for _, row in df.iterrows():
        path = row['image_backup_url']
        label = row['label']
        path_label_list.append((path, label))
    return path_label_list

path_label = create_path_label_list(data2)
print(path_label[0:3])

[('https://lab.plantnet.org/LifeCLEF/PlantCLEF2024/single_plant_training_data/PlantCLEF2024singleplanttrainingdata/train/1396710/59feabe1c98f06e7f819f73c8246bd8f1a89556b.jpg', 4826), ('https://lab.plantnet.org/LifeCLEF/PlantCLEF2024/single_plant_training_data/PlantCLEF2024singleplanttrainingdata/train/1396710/dc273995a89827437d447f29a52ccac86f65476e.jpg', 4826), ('https://lab.plantnet.org/LifeCLEF/PlantCLEF2024/single_plant_training_data/PlantCLEF2024singleplanttrainingdata/train/1396710/416235e7023a4bd1513edf036b6097efc693a304.jpg', 4826)]


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, path_label, transform=None):
        self.path_label = path_label
        self.transform = transform

    def __len__(self):
        return len(self.path_label)


    def __getitem__(self, idx):
        path, label = self.path_label[idx]

        try:
            # If the path starts with 'http' or contains '://', it's a URL
            if path.startswith('http') or '://' in path:
                import requests
                from io import BytesIO

                response = requests.get(path, timeout=10)
                if response.status_code == 200:
                    img = Image.open(BytesIO(response.content)).convert('RGB')
                else:
                    print(f"Warning: Could not fetch image from {path}, status code: {response.status_code}")
                    img = self._get_dummy_image()
            else:
                # Normal local file handling
                img = Image.open(path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {path}: {str(e)}")
            img = self._get_dummy_image()

        if self.transform is not None:
            img = self.transform(img)

        pil_to_tensor = transforms.ToTensor()
        resize = transforms.Resize((224, 224))
        img = pil_to_tensor(img)
        img = resize(img)
        print(img.shape)
        return img, label

    def _get_dummy_image(self):

        return Image.new('RGB', (224, 224), color=(0, 0, 0))

In [ ]:
dataset = CustomDataset(path_label)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
# Iterate through DataLoader
for batch_idx, (inputs, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx}:")
    print(f"Inputs:\n{inputs.shape}")
    print(f"Labels:\n{labels}")
    print('-' * 30)
    break


torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
Batch 0:
Inputs:
torch.Size([2, 3, 224, 224])
Labels:
tensor([6011, 4321])
------------------------------
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
Batch 1:
Inputs:
torch.Size([2, 3, 224, 224])
Labels:
tensor([2361, 3707])
------------------------------
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
Batch 2:
Inputs:
torch.Size([2, 3, 224, 224])
Labels:
tensor([3833, 5373])
------------------------------
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
Batch 3:
Inputs:
torch.Size([2, 3, 224, 224])
Labels:
tensor([1242, 5001])
------------------------------
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
Batch 4:
Inputs:
torch.Size([2, 3, 224, 224])
Labels:
tensor([1189, 6974])
------------------------------
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
Batch 5:
Inputs:
torch.Size([2, 3, 224, 224])
Labels:
tensor([3908, 7726])
------------------------------
torch.Size([3, 224, 224])
torch.Size([3, 224, 224])
